In [1]:
import pandas as pd
import cv2
import os
import numpy as np
import subprocess
from pathlib import Path

# ========== 1. НАСТРОЙКИ ==========
TRAIN_FOLDER = 'train'
CSV_FILE = 'annotation_car_plate_train.csv'
OUTPUT_DIR = 'cascade_training'
POSITIVE_FILE = 'positive.txt'
NEGATIVE_FILE = 'negative.txt'
VEC_FILE = 'positive.vec'
CASCADE_DIR = 'cascade'

# Параметры каскада (пропорции номерного знака)
CASCADE_WIDTH = 48
CASCADE_HEIGHT = 16

# Создание директорий
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'negative_images'), exist_ok=True)
os.makedirs(CASCADE_DIR, exist_ok=True)

# ========== 2. ЧТЕНИЕ И АНАЛИЗ ДАННЫХ ==========
print("="*60)
print("ПОДГОТОВКА ДАННЫХ ДЛЯ ОБУЧЕНИЯ КАСКАДА ХААРА")
print("="*60)

df = pd.read_csv(CSV_FILE)
print(f"\nЗагружено записей: {len(df)}")
print(f"Уникальных изображений: {df['file'].nunique()}")
print(f"\nПример данных:")
print(df.head())

# ========== 3. ОПРЕДЕЛЕНИЕ РАСШИРЕНИЯ ФАЙЛОВ ==========
def find_image_extension(sample_file, train_folder):
    """Определяет расширение файлов изображений"""
    for ext in ['.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG']:
        if os.path.exists(os.path.join(train_folder, sample_file + ext)):
            return ext
    return '.jpg'  # по умолчанию

IMAGE_EXT = find_image_extension(df['file'].iloc[0], TRAIN_FOLDER)
print(f"\nОпределено расширение файлов: {IMAGE_EXT}")

# ========== 4. СОЗДАНИЕ ПОЗИТИВНЫХ ПРИМЕРОВ ==========
def create_positive_file(df, train_folder, output_file, image_ext):
    """
    Создает файл с описанием позитивных примеров
    Формат: путь/к/файлу.jpg количество_объектов x y width height
    """
    print(f"\n[1/4] Создание файла позитивных примеров...")
    
    valid_count = 0
    invalid_count = 0
    
    with open(output_file, 'w') as f:
        for idx, row in df.iterrows():
            img_name = row['file'] + image_ext
            img_path = os.path.join(train_folder, img_name)
            
            # Проверка существования файла
            if not os.path.exists(img_path):
                invalid_count += 1
                continue
            
            # Координаты номерного знака
            x = int(row['xmin'])
            y = int(row['ymin'])
            width = int(row['xmax'] - row['xmin'])
            height = int(row['ymax'] - row['ymin'])
            
            # Проверка корректности координат
            if width <= 0 or height <= 0:
                invalid_count += 1
                continue
            
            # Запись в файл
            f.write(f"{img_path} 1 {x} {y} {width} {height}\n")
            valid_count += 1
    
    print(f"✓ Создано позитивных примеров: {valid_count}")
    print(f"✗ Пропущено (файл не найден/некорректные координаты): {invalid_count}")
    
    return valid_count

num_positive = create_positive_file(df, TRAIN_FOLDER, POSITIVE_FILE, IMAGE_EXT)

if num_positive == 0:
    print("\n❌ ОШИБКА: Не найдено ни одного валидного изображения!")
    print(f"Проверьте, что изображения находятся в папке '{TRAIN_FOLDER}'")
    exit(1)

# ========== 5. СОЗДАНИЕ НЕГАТИВНЫХ ПРИМЕРОВ ==========
def create_negative_samples(df, train_folder, image_ext, output_folder, 
                           output_file, num_negatives=2000):
    """
    Создает негативные примеры (области изображения без номеров)
    """
    print(f"\n[2/4] Создание негативных примеров (цель: {num_negatives})...")
    
    negative_folder = os.path.join(OUTPUT_DIR, output_folder)
    os.makedirs(negative_folder, exist_ok=True)
    
    negative_count = 0
    processed_images = 0
    
    with open(output_file, 'w') as f:
        for idx, row in df.iterrows():
            if negative_count >= num_negatives:
                break
                
            img_name = row['file'] + image_ext
            img_path = os.path.join(train_folder, img_name)
            
            if not os.path.exists(img_path):
                continue
                
            img = cv2.imread(img_path)
            if img is None:
                continue
            
            processed_images += 1
            h, w = img.shape[:2]
            
            # Координаты номерного знака
            plate_x = int(row['xmin'])
            plate_y = int(row['ymin'])
            plate_w = int(row['xmax'] - row['xmin'])
            plate_h = int(row['ymax'] - row['ymin'])
            
            # Создаем 3-5 негативных патчей из каждого изображения
            patches_per_image = min(5, (num_negatives - negative_count) // max(1, (len(df) - idx)))
            
            for i in range(patches_per_image):
                # Случайный размер патча
                neg_w = np.random.randint(max(50, plate_w//2), min(w, plate_w*3))
                neg_h = np.random.randint(max(30, plate_h//2), min(h, plate_h*3))
                
                # Попытки найти область без пересечения с номером
                for attempt in range(10):
                    neg_x = np.random.randint(0, max(1, w - neg_w))
                    neg_y = np.random.randint(0, max(1, h - neg_h))
                    
                    # Проверка на пересечение с номером
                    if (neg_x + neg_w < plate_x or neg_x > plate_x + plate_w or
                        neg_y + neg_h < plate_y or neg_y > plate_y + plate_h):
                        
                        neg_patch = img[neg_y:neg_y+neg_h, neg_x:neg_x+neg_w]
                        
                        if neg_patch.size > 0 and neg_patch.shape[0] > 10 and neg_patch.shape[1] > 10:
                            neg_filename = f"negative_{negative_count:05d}.jpg"
                            neg_path = os.path.join(negative_folder, neg_filename)
                            cv2.imwrite(neg_path, neg_patch)
                            f.write(f"{neg_path}\n")
                            negative_count += 1
                            break
                
                if negative_count >= num_negatives:
                    break
            
            if processed_images % 100 == 0:
                print(f"  Обработано изображений: {processed_images}, создано негативов: {negative_count}")
    
    print(f"✓ Создано негативных примеров: {negative_count}")
    return negative_count

num_negative = create_negative_samples(
    df, TRAIN_FOLDER, IMAGE_EXT, 
    'negative_images', NEGATIVE_FILE, 
    num_negatives=min(num_positive * 2, 3000)
)

# ========== 6. СОЗДАНИЕ VEC ФАЙЛА ==========
def create_vec_file(positive_file, vec_file, width, height):
    """
    Создает .vec файл из позитивных примеров
    """
    print(f"\n[3/4] Создание .vec файла...")
    
    num_samples = sum(1 for _ in open(positive_file))
    
    cmd = [
        'opencv_createsamples',
        '-info', positive_file,
        '-vec', vec_file,
        '-w', str(width),
        '-h', str(height),
        '-num', str(num_samples)
    ]
    
    print(f"Команда: {' '.join(cmd)}")
    print(f"Количество образцов: {num_samples}")
    
    try:
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=300)
        
        if result.returncode == 0:
            print(f"✓ Файл {vec_file} успешно создан")
            return True
        else:
            print(f"❌ Ошибка создания .vec файла:")
            print(result.stderr)
            return False
    except FileNotFoundError:
        print("❌ Утилита opencv_createsamples не найдена!")
        print("Установите OpenCV с инструментами обучения.")
        return False
    except subprocess.TimeoutExpired:
        print("❌ Timeout при создании .vec файла")
        return False

vec_success = create_vec_file(POSITIVE_FILE, VEC_FILE, CASCADE_WIDTH, CASCADE_HEIGHT)

if not vec_success:
    print("\n⚠️  Не удалось создать .vec файл. Проверьте установку OpenCV.")
    exit(1)

# ========== 7. ОБУЧЕНИЕ КАСКАДА ==========
def train_cascade(vec_file, negative_file, cascade_dir, num_pos, num_neg,
                 num_stages=15, width=48, height=16):
    """
    Обучает каскад Хаара
    """
    print(f"\n[4/4] Обучение каскада Хаара...")
    print(f"⚠️  Это может занять от нескольких часов до суток!")
    print(f"\nПараметры обучения:")
    print(f"  - Позитивных примеров: {num_pos} (используется {int(num_pos * 0.85)})")
    print(f"  - Негативных примеров: {num_neg}")
    print(f"  - Количество стадий: {num_stages}")
    print(f"  - Размер детектора: {width}x{height}")
    
    cmd = [
        'opencv_traincascade',
        '-data', cascade_dir,
        '-vec', vec_file,
        '-bg', negative_file,
        '-numPos', str(int(num_pos * 0.85)),  # 85% от позитивных примеров
        '-numNeg', str(num_neg),
        '-numStages', str(num_stages),
        '-w', str(width),
        '-h', str(height),
        '-minHitRate', '0.999',
        '-maxFalseAlarmRate', '0.5',
        '-mode', 'ALL',
        '-featureType', 'HAAR',
        '-precalcValBufSize', '2048',
        '-precalcIdxBufSize', '2048'
    ]
    
    print(f"\nКоманда: {' '.join(cmd)}")
    print("\nНачало обучения...\n")
    
    try:
        # Запуск с выводом в реальном времени
        process = subprocess.Popen(cmd, stdout=subprocess.PIPE, 
                                  stderr=subprocess.STDOUT, text=True)
        
        for line in process.stdout:
            print(line, end='')
        
        process.wait()
        
        if process.returncode == 0:
            print("\n" + "="*60)
            print("✓ ОБУЧЕНИЕ ЗАВЕРШЕНО УСПЕШНО!")
            print(f"✓ Каскад сохранен в {cascade_dir}/cascade.xml")
            print("="*60)
            return True
        else:
            print(f"\n❌ Ошибка обучения (код возврата: {process.returncode})")
            return False
            
    except FileNotFoundError:
        print("❌ Утилита opencv_traincascade не найдена!")
        print("Установите OpenCV с инструментами обучения.")
        return False

# Запуск обучения
train_success = train_cascade(
    VEC_FILE, NEGATIVE_FILE, CASCADE_DIR,
    num_positive, num_negative,
    num_stages=15,
    width=CASCADE_WIDTH,
    height=CASCADE_HEIGHT
)

# ========== 8. ТЕСТИРОВАНИЕ КАСКАДА ==========
if train_success:
    print("\n" + "="*60)
    print("ТЕСТИРОВАНИЕ ОБУЧЕННОГО КАСКАДА")
    print("="*60)
    
    def test_cascade_on_samples(cascade_path, df, train_folder, image_ext, num_samples=10):
        """
        Тестирует каскад на случайных примерах из датасета
        """
        cascade = cv2.CascadeClassifier(cascade_path)
        
        if cascade.empty():
            print("❌ Ошибка загрузки каскада!")
            return
        
        print(f"\nТестирование на {num_samples} случайных изображениях...\n")
        
        # Выбираем случайные изображения
        sample_indices = np.random.choice(len(df), min(num_samples, len(df)), replace=False)
        
        results_dir = 'test_results'
        os.makedirs(results_dir, exist_ok=True)
        
        detected_count = 0
        
        for i, idx in enumerate(sample_indices):
            row = df.iloc[idx]
            img_name = row['file'] + image_ext
            img_path = os.path.join(train_folder, img_name)
            
            if not os.path.exists(img_path):
                continue
            
            img = cv2.imread(img_path)
            if img is None:
                continue
            
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            # Детектирование
            plates = cascade.detectMultiScale(
                gray,
                scaleFactor=1.05,
                minNeighbors=3,
                minSize=(30, 10),
                maxSize=(400, 150)
            )
            
            # Истинная позиция номера
            true_x = int(row['xmin'])
            true_y = int(row['ymin'])
            true_w = int(row['xmax'] - row['xmin'])
            true_h = int(row['ymax'] - row['ymin'])
            
            # Отрисовка истинного положения (зеленый)
            cv2.rectangle(img, (true_x, true_y), 
                         (true_x + true_w, true_y + true_h), 
                         (0, 255, 0), 2)
            cv2.putText(img, 'TRUE', (true_x, true_y-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
            # Отрисовка детекций (красный)
            for (x, y, w, h) in plates:
                cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 2)
                cv2.putText(img, 'DETECTED', (x, y-10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                detected_count += 1
            
            # Сохранение результата
            output_path = os.path.join(results_dir, f'test_{i:03d}_{row["file"]}.jpg')
            cv2.imwrite(output_path, img)
            
            print(f"  [{i+1}/{num_samples}] {img_name}: найдено {len(plates)} номер(ов)")
        
        print(f"\n✓ Результаты сохранены в папке '{results_dir}'")
        print(f"✓ Всего детекций: {detected_count}")
    
    cascade_xml = os.path.join(CASCADE_DIR, 'cascade.xml')
    if os.path.exists(cascade_xml):
        test_cascade_on_samples(cascade_xml, df, TRAIN_FOLDER, IMAGE_EXT, num_samples=20)
    else:
        print(f"⚠️  Файл каскада не найден: {cascade_xml}")

print("\n" + "="*60)
print("ЗАВЕРШЕНО")
print("="*60)

ПОДГОТОВКА ДАННЫХ ДЛЯ ОБУЧЕНИЯ КАСКАДА ХААРА

Загружено записей: 482
Уникальных изображений: 482

Пример данных:
   xmin  ymin  xmax  ymax   name      file  width  height  class     Xcent  \
0  1869  1678  2153  1757  plate  img_2297   4032    3024    NaN  0.498760   
1  1826  1493  1972  1535  plate  img_2745   4032    3024    NaN  0.470982   
2  1804  1510  1983  1556  plate  img_1707   4032    3024    NaN  0.469618   
3  1727  1733  2065  1824  plate  img_2665   4032    3024    NaN  0.470238   
4  1957  1694  2189  1746  plate  img_1712   4032    3024    NaN  0.514137   

      Ycent      boxW      boxH  
0  0.567956  0.070437  0.026124  
1  0.500661  0.036210  0.013889  
2  0.506944  0.044395  0.015212  
3  0.588128  0.083829  0.030093  
4  0.568783  0.057540  0.017196  

Определено расширение файлов: .jpg

[1/4] Создание файла позитивных примеров...
✓ Создано позитивных примеров: 482
✗ Пропущено (файл не найден/некорректные координаты): 0

[2/4] Создание негативных примеров (цель:

In [4]:
import cv2

CASCADE_PATH = 'cascade/cascade.xml'
TEST_IMAGE = 'test_image.jpg'  # или любое другое

def test_single_image(image_path, cascade_path):
    """
    Тестирует каскад на одном изображении
    """
    # Загрузка каскада
    cascade = cv2.CascadeClassifier(cascade_path)
    
    if cascade.empty():
        print(f"❌ Не удалось загрузить каскад: {cascade_path}")
        return
    
    # Загрузка изображения
    img = cv2.imread(image_path)
    if img is None:
        print(f"❌ Не удалось загрузить изображение: {image_path}")
        return
    
    print(f"Тестирование на изображении: {image_path}")
    print(f"Размер изображения: {img.shape[1]}x{img.shape[0]}")
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Детектирование с разными параметрами
    print("\nПодбор параметров детектирования...")
    
    for scale_factor in [1.05, 1.1, 1.15]:
        for min_neighbors in [3, 5, 7]:
            plates = cascade.detectMultiScale(
                gray,
                scaleFactor=scale_factor,
                minNeighbors=min_neighbors,
                minSize=(30, 10),
                maxSize=(500, 200)
            )
            
            if len(plates) > 0:
                print(f"  scaleFactor={scale_factor}, minNeighbors={min_neighbors}: "
                      f"найдено {len(plates)} объект(ов)")
    
    # Детектирование с лучшими параметрами
    plates = cascade.detectMultiScale(
        gray,
        scaleFactor=1.05,
        minNeighbors=5,
        minSize=(50, 20),
        maxSize=(350, 120)
    )
    
    print(f"\n✓ Найдено номерных знаков: {len(plates)}")
    
    # Отрисовка результатов
    for i, (x, y, w, h) in enumerate(plates):
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 3)
        cv2.putText(img, f'Plate {i+1}', (x, y-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        print(f"  Номер {i+1}: x={x}, y={y}, w={w}, h={h}")
    
    # Сохранение результата
    output_path = 'detection_result.jpg'
    cv2.imwrite(output_path, img)
    print(f"\n✓ Результат сохранен: {output_path}")
    
    # Отображение (если запущено локально)
    cv2.imshow('Detection Result', cv2.resize(img, (800, 600)))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    test_single_image(TEST_IMAGE, CASCADE_PATH)

Тестирование на изображении: test_image.jpg
Размер изображения: 800x600

Подбор параметров детектирования...
  scaleFactor=1.05, minNeighbors=3: найдено 2 объект(ов)
  scaleFactor=1.05, minNeighbors=5: найдено 1 объект(ов)
  scaleFactor=1.05, minNeighbors=7: найдено 1 объект(ов)
  scaleFactor=1.1, minNeighbors=3: найдено 1 объект(ов)
  scaleFactor=1.1, minNeighbors=5: найдено 1 объект(ов)
  scaleFactor=1.1, minNeighbors=7: найдено 1 объект(ов)
  scaleFactor=1.15, minNeighbors=3: найдено 2 объект(ов)
  scaleFactor=1.15, minNeighbors=5: найдено 2 объект(ов)
  scaleFactor=1.15, minNeighbors=7: найдено 1 объект(ов)

✓ Найдено номерных знаков: 1
  Номер 1: x=267, y=295, w=120, h=40

✓ Результат сохранен: detection_result.jpg
